# Pulse | ONDC Pincode-Merchant mapping
#### `ONDC Build for Bharat Hackathon`

**Author:** Sarang Galada<br>
**Email ID:** sarang.galada@gmail.com<br>
**Date created:** 07/02/2024<br><br>
**Abstract:** Due to its democratized nature, the Open Network for Digital Commerce (ONDC) is approximated to have upwards of 100 million merchants onboarded on the platform! Thus, a matrix having all Indian pincodes (\~19,300) as columns and merchants (\~100m) as rows, will both be colossal and extremely sparse.

Now imagine a customer-facing app having to retrieve all the merchants servicing a given PIN Code in near-instant time. This calls for the development of efficient algorithms and data structures to optimally store and retrieve Pincode-Merchant mappings.

In this solution, Radix Trees are used as the data structure. They are optimal for our particular use-case since:
1. They don't store redundant information (ie. 100% density) unlike sparse matrices
2. Offer exceptional compaction by exploiting patterns in the PIN Code structure
3. Not only do they offer constant-time O(1) lookup (which we are most concerned about), but even that constant-time taken is minimum compared to other O(1)-lookup data structures available. Hence retrieval time is near-instant (in micro/milli seconds!) irrespective of data dimensions
4. Extreme flexibility. Can handle:
		- Any number of PIN Codes and Merchants
		- On-the-fly updation of PIN Codes and Merchants
		- Variable-length PIN Codes, making them scalable across other countries too.
		- Easily interconvertible to/from standard formats such as CSV
		- Can be applied to a host of analogous problem statements (eg. mappings between Merchant & Product Category, Companies & Job Applicants, Universities & Candidates)
  
<br>

*   *Hackathon*: `Build for Bharat, 2024 by ONDC`
*   *Problem Statement*: `Optimal storage & retrieval in m*n sparse matrix`
*   *Scale of the solution*: `19,300 PIN Codes (total in India) and 1,00,000 Merchants`
*   *Data Structure used*: `Radix Tree`

In [ ]:
## Loading libraries

import numpy as np
import pandas as pd
import random
import time
import pickle

## Architecture design

In [ ]:
## Defining the Radix Tree Class

class RadixTreeNode:
    def __init__(self):
        self.children = {}
        self.merchant_ids = set()

class RadixTree:
    def __init__(self):
        self.root = RadixTreeNode()

    def insert(self, pincode, merchant_id):
        pincode = str(pincode)
        node = self.root
        for digit in pincode:
            if digit not in node.children:
                node.children[digit] = RadixTreeNode()
            node = node.children[digit]
        node.merchant_ids.add(merchant_id)

    def load_merchant(self, merchant_pin_array, merchant_id):
      for pincode in merchant_pin_array:
        self.insert(pincode, merchant_id)

    def search(self, pincode):
        node = self.root
        for digit in pincode:
            if digit not in node.children:
                return set()  # ie. Pincode not found
            node = node.children[digit]
        return node.merchant_ids

    def print_tree(self):
            self._print_node(self.root, "")

    def _print_node(self, node, prefix):
        if node.merchant_ids:
            print(prefix + " -> " + str(node.merchant_ids))
        for digit, child in node.children.items():
            self._print_node(child, prefix + digit)

## Data preparation & storage

In [ ]:
## Loading an array containing all 19300 existing Indian pincodes (as of Jan 2024)

pincodes = pd.read_csv("data/unique indian pincodes.csv", header=None)
pincodes = pincodes.values.reshape(-1)
total_pincodes = len(pincodes)
total_pincodes

19300

### Option 1


In [ ]:
## Option 1: Generate 100k merchants, each servicing 50-1000 PIN codes, and load them onto a Radix Tree

radix_tree = RadixTree()

t0 = time.time()

for i in range(100000):
  # Randomly assign a bunch of 50-1000 unsorted pincodes from the pool to each merchant
  st = random.randint(0, total_pincodes-1000) # start index
  num_pins = random.randint(50, 1000) # chunk size
  merchant_pins = pincodes[st:st+num_pins] # grab a chunk and assign

  radix_tree.load_merchant(merchant_pins, i)

t1 = time.time()

# Save the Radix Tree object to a binary file
with open('data/100k_merchants.pkl', 'wb') as f:
    pickle.dump(radix_tree, f)

# Print the radix tree
# radix_tree.print_tree()

print(f"Radix Tree building time: {t1-t0} seconds")

In [ ]:
## Option 1: Load the pre-built Radix Tree

t0 = time.time()

# Load the object from the binary file
with open('data/100k_merchants.pkl', 'rb') as f:
    radix_tree = pickle.load(f)

t1 = time.time()

print(f"Radix Tree loading time: {t1-t0} seconds")

File loading time: 11.696804523468018 seconds


### Option 2

In [ ]:
## Option 2: Generate 100k merchants, each servicing 50-1000 PIN codes, and load them onto a CSV file

df = pd.DataFrame(0, index=range(1000), columns=range(100000))

t0 = time.time()

for i in range(100000):
    # Randomly assign a bunch of 50-1000 unsorted pincodes from the pool to each merchant
    st = np.random.randint(0, total_pincodes-1000) # start index
    serviceable_size = np.random.randint(50, 1000) # chunk size
    serviceable_pins = np.concatenate([pincodes[st:st+serviceable_size], np.zeros(1000-serviceable_size, dtype=int)]) # grab a chunk and assign

    # Add merchant's id as column name and serviceable pincodes as column values
    df[i] = serviceable_pins

t1 = time.time()

df.to_csv("data/10k_merchants.csv", index=False)

print(f"CSV file building time: {t1-t0} seconds")

In [ ]:
## Option 2: Load the CSV file and building a Radix Tree from it

# Function to convert from CSV format to Radix Tree object
def csv_to_radix_tree(csv):
    df = pd.read_csv(csv)
    tree = RadixTree()
    for i in df.columns:
        tree.load_merchant(df[i].values, i)
    return tree

t0 = time.time()

radix_tree = csv_to_radix_tree(pd.read_csv('data/10k_merchants.csv'))

t1 = time.time()

print(f"Radix Tree building time: {t1-t0} seconds")

## Testing & demo

In [ ]:
## Working Example (Buyer app side): Retrieving all merchants servicing a specific pincode

pincode = "140507"

t0 = time.time()

# Searching the Radix Tree for the pincode
merchants = radix_tree.search(pincode)
t1 = time.time()

print(f"IDs of Merchants servicing pincode {pincode}: {merchants}")
print(f"No. of Merchants servicing pincode {pincode}: {len(merchants)}")

print(f"\nRetrieval time: {t1-t0} seconds!!!")

IDs of Merchants servicing pincode 140507: {24577, 40969, 73738, 90130, 81954, 81964, 90163, 8244, 49223, 8264, 98376, 98397, 57437, 8294, 90215, 24681, 82028, 41072, 49270, 90244, 73864, 41101, 32914, 41106, 32920, 49312, 8357, 90282, 90289, 24766, 16575, 16576, 24771, 41157, 24776, 90314, 65740, 65753, 32986, 16609, 49378, 82151, 49390, 33011, 65780, 73973, 24824, 82171, 259, 57611, 8460, 16659, 98579, 74017, 90401, 65828, 65832, 49453, 24885, 16698, 57660, 33085, 90431, 324, 49478, 57672, 41291, 98637, 82254, 49491, 98647, 65890, 98664, 16748, 24943, 49533, 98702, 98704, 49555, 74131, 82323, 57752, 49569, 423, 41400, 41412, 57798, 82381, 74196, 25050, 25058, 82416, 8695, 90623, 33282, 16905, 16911, 8722, 82457, 16929, 25123, 66093, 98861, 98865, 82484, 90682, 74308, 49742, 41551, 74319, 90702, 25172, 57953, 8809, 66155, 33389, 49773, 98927, 82547, 74356, 90746, 82557, 25215, 41599, 82564, 82566, 648, 33418, 57996, 90769, 98968, 41625, 58010, 74412, 49837, 8878, 74416, 74423, 58048, 

In [ ]:
## Working Example 2

pincode = "110001"

t0 = time.time()
merchants = radix_tree.search(pincode)
t1 = time.time()

print(f"IDs of Merchants servicing pincode {pincode}: {merchants}")
print(f"No. of Merchants servicing pincode {pincode}: {len(merchants)}")

print(f"\nRetrieval time: {t1-t0} seconds!!!")

IDs of Merchants servicing pincode 110001: {65542, 49161, 98315, 24589, 90126, 16402, 27, 24611, 24613, 73766, 16448, 16449, 90176, 90189, 57428, 82006, 49242, 98398, 41056, 16487, 105, 8304, 73840, 41079, 82046, 32895, 129, 73861, 41102, 57494, 98455, 98458, 57508, 98476, 41139, 201, 213, 98518, 32983, 90333, 65767, 98543, 57584, 244, 90358, 90360, 33036, 41230, 8475, 57627, 8489, 16682, 57650, 33077, 33080, 8505, 41272, 49465, 57675, 8526, 82255, 8529, 65874, 82264, 33114, 49506, 356, 57701, 90468, 57713, 8568, 8571, 49532, 74115, 82310, 82315, 49553, 82321, 16787, 90516, 74133, 8602, 41374, 24991, 98722, 82347, 33200, 82355, 25012, 16841, 41418, 41420, 41421, 98771, 25055, 25056, 74208, 90622, 49665, 57869, 66061, 98834, 8732, 74270, 90662, 25129, 74294, 579, 90691, 588, 74316, 82516, 8795, 49756, 66141, 57950, 98909, 33379, 90726, 33384, 90729, 98928, 66161, 635, 74363, 90748, 57983, 66181, 17035, 669, 33444, 41636, 58024, 82600, 8875, 74414, 688, 66228, 25271, 66236, 74429, 58047,